Train CDSSM neural netword

In [ ]:
%%bash
# Run this code on GPU server
python train_cdssm_tele2.py --cuda=1 --epoch 40 --dropout 0.4 --seed 46\
                      --netsize 200 --parallel 10\
                      --run tele2_cdssm\
                      --restore-model ../data/models/Matcher_best_clear.pth\
                      --train-data ../data/tele2/train_data.tsv\
                      --valid-data ../data/tele2/valid_data.tsv\
                      --save-every 3 --batch-size 50\
                      --lr 0.00002 --patience 4 --emb-size 300\
                      --emb-path '../data/araneum_none_fasttextskipgram_300_5_2018.model' |& tee run_cdssm_tele2.sh.log


In [20]:
%%bash
cd DSSM;
echo "   Positive examples"
python predict_tele2.py --model ../data/Matcher_best_clear.pth\
                           --mtx-path ../data/ft_vect.dat\
                           --meta-path ../data/ft_meta.json\
                           --input-file ../data/pos_example.tsv | tail -n +4

echo "   Negative examples"
python predict_tele2.py --model ../data/Matcher_best_clear.pth\
                           --mtx-path ../data/ft_vect.dat\
                           --meta-path ../data/ft_meta.json\
                           --input-file ../data/neg_example.tsv | tail -n +4

   Positive examples
0.299609899520874
0.3806736469268799
0.08130992949008942
0.5117695927619934
0.21419133245944977
0.2816021740436554
0.07308541238307953
0.43856534361839294
0.3057277500629425
0.2919294238090515
   Negative examples
0.0008508919854648411
0.14637349545955658
0.0018008145270869136
0.035722069442272186
0.0014386859256774187
0.10372891277074814
0.30649563670158386
0.00030640693148598075
0.16311630606651306
0.06232735514640808


### Generate question vectors

In [19]:
%%bash
# Collect all questions
cat ./data/train_data.tsv ./data/valid_data.tsv | cut -f 1 | sort | uniq | grep -E '[a-zа-яА-ЯА-Я]+' > ./data/all_questions.txt
echo "Collect questions: Done!"
cd DSSM;
python predict_tele2_qv.py --model ../data/Matcher_best_clear.pth\
                           --mtx-path ../data/ft_vect.dat\
                           --meta-path ../data/ft_meta.json\
                           --input-file ../data/all_questions.txt | tail -n +4 > ../data/question_vectors.tsv
head -n 1 ../data/question_vectors.tsv

Collect questions: Done!
-0.9552842974662781 -0.9921454191207886 -0.9999502301216125 -0.9994466304779053 0.022762859240174294 0.991820752620697 0.9997281432151794 0.6344990730285645 -0.638560950756073 0.9084559082984924 0.4167441427707672 -0.4504954516887665 -0.9595537781715393 0.08190415799617767 -0.9998862147331238 0.9998322129249573 0.5831775665283203 0.9057847857475281 -0.9999468922615051 0.682098925113678 0.9999850988388062 0.27462974190711975 0.8360445499420166 0.012333379127085209 -0.3065294325351715 -0.019926398992538452 0.9999061822891235 0.9999964237213135 0.8302494287490845 0.884909987449646 -0.3489568531513214 0.7170425057411194 0.998370885848999 -0.9817487597465515 0.9998788833618164 -0.08828888833522797 0.12075190991163254 0.9992536902427673 0.9200456142425537 -0.9947767853736877 -0.3379512429237366 0.9999749064445496 -0.005467731039971113 -0.6138373613357544 0.16392555832862854 0.982033908367157 -0.6082790493965149 -0.12081235647201538 0.11605052649974823 -0.687864959239

### Generate answer vectors

In [ ]:
%%bash
cd DSSM;
python predict_tele2_av.py --model ../data/Matcher_best_clear.pth\
                           --mtx-path ../data/ft_vect.dat\
                           --meta-path ../data/ft_meta.json\
                           --input-file ../data/pos_example.tsv | tail -n +4 > ../data/question_vectors.tsv